In [24]:
from IPython.display import Image
Image(filename='C:\\Users\\Harsh\\Anaconda3\\Notebooks\\CIS_8005_Project\\home.jpeg')

# Sentiment Analysis using Twitter Data

## As part of CIS 8005 Final Project, we have chosen to perform social network analytics on Twitter feed data. We will analyze the twitter text anf calculate its sentimental value. This way, we can determine if a user has shown a positive, negative or neutral sentiment.

### Step 1: There are two appraoches to retrieve twitter data :
### 1 ) Easy one : Download it online
### 2 ) Read it from Twitter Streaming API

In [ ]:
#Import the necessary methods from tweepy library
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API 
access_token = "633931786-7bNaORmqomwFOLcb8VGnoKdrymudKLMdI1RQ5BQ1"
access_token_secret = "eI6Yr92641L69MY7u5f6ecWUGrVVKfJ50xNUEMPbWgcRU"
consumer_key = "0yDf9lAFzpbAWhaVMYLS1xyyW"
consumer_secret = "7TXBikm0MyVlQro7U9saQHR6dMY3tNAWNUZWuubjNGVJtQtcL8"

class StdOutListener(tweepy.StreamListener):

    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)

def main():
    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    stream.filter(track=['game of thrones', 'Game of Thrones', 'GOT6', 'GoT6'])

main()

### This raw data is stored in MongoDB in JSON format. We have also used two external tables which contain time zone data and english word dictionary data for cleaning and analysis.

In [13]:
from pymongo import MongoClient
import html.parser
import pandas as pd
import numpy as np
import sklearn.feature_extraction.text as ext
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords      
import nltk.data   
from nltk.stem.porter import PorterStemmer
import time
import re

### Fetching data from MongoDB using PyMongo module and storing it as Pandas DataFrame

In [14]:
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
time_zone = pd.io.parsers.read_csv("C:\\Users\\Harsh\\Documents\\Twitter\\time_zone_map.csv")
client = MongoClient()
db = client['TwitterSentimentAnalysis']
collection = db['Tweets_Raw']
rawTweets  = pd.DataFrame(list(collection.find()))
rawTweets.head(10)

,_id,contributors,coordinates,created_at,entities,favorite_count,favorited,filter_level,geo,id,...,lang,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,571c1cac3f712e0c4fcabece,None,None,Mon Feb 17 12:20:56 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388391811997696,...,tl,None,False,0,False,NaN,"<a href=""http://www.facebook.com/twitter"" rel=...",Sabi nung isang tao dito nung shinutdown si ro...,False,{'profile_banner_url': 'https://pbs.twimg.com/...
1,571c1cac3f712e0c4fcabecf,None,"{'type': 'Point', 'coordinates': [107.00031148...",Mon Feb 17 12:21:06 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,"{'type': 'Point', 'coordinates': [-6.22606633,...",435388432211525632,...,in,"{'contained_within': [], 'country': 'Indonesia...",False,0,False,NaN,"<a href=""http://foursquare.com"" rel=""nofollow""...",ROBOCOP B) (@ XXI Summarecon Mal Bekasi) http:...,False,{'profile_banner_url': 'https://pbs.twimg.com/...
2,571c1cac3f712e0c4fcabed0,None,None,Mon Feb 17 12:21:10 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388451060322304,...,in,None,NaN,0,False,NaN,"<a href=""http://blackberry.com/twitter"" rel=""n...",Robocop =D ga jadiwess ..,False,{'profile_banner_url': 'https://pbs.twimg.com/...
3,571c1cac3f712e0c4fcabed1,None,None,Mon Feb 17 12:21:12 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388458362630144,...,en,None,NaN,0,False,NaN,"<a href=""http://blackberry.com/twitter"" rel=""n...",Watching Robocop with my baby bro,False,{'profile_banner_url': 'https://pbs.twimg.com/...
4,571c1cac3f712e0c4fcabed2,None,None,Mon Feb 17 12:21:11 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388456366116865,...,ar,None,NaN,0,False,NaN,web,أمس شفت Robocop .. طلع أفضل مما توقعته .. فلم ...,False,{'profile_banner_url': 'https://pbs.twimg.com/...
5,571c1cac3f712e0c4fcabed3,None,None,Mon Feb 17 12:21:25 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388514851495936,...,en,None,NaN,0,False,NaN,web,istrine Alex Murphy cakep banget #RoboCop,False,{'profile_banner_url': 'https://pbs.twimg.com/...
6,571c1cac3f712e0c4fcabed4,None,None,Mon Feb 17 12:21:16 +0000 2014,"{'symbols': [], 'user_mentions': [{'indices': ...",0,False,medium,None,435388477463474176,...,in,None,NaN,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@erraaisya dok dap op cite robocop,False,{'profile_banner_url': 'https://pbs.twimg.com/...
7,571c1cac3f712e0c4fcabed5,None,None,Mon Feb 17 12:21:30 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388533872676864,...,en,None,False,0,False,NaN,"<a href=""https://path.com/"" rel=""nofollow"">Pat...","Watching RoboCop (with Adila, Muhammad , and 6...",False,{'profile_banner_url': 'https://pbs.twimg.com/...
8,571c1cac3f712e0c4fcabed6,None,None,Mon Feb 17 12:21:34 +0000 2014,"{'symbols': [], 'user_mentions': [], 'urls': [...",0,False,medium,None,435388552738648064,...,en,None,False,0,False,NaN,"<a href=""https://path.com/"" rel=""nofollow"">Pat...",Watching RoboCop (with Bayu at Puri XXI) — htt...,False,"{'profile_sidebar_border_color': '65B0DA', 'de..."
9,571c1cac3f712e0c4fcabed7,None,None,Mon Feb 17 12:21:43 +0000 2014,"{'symbols': [], 'user_mentions': [{'indices': ...",0,False,medium,None,435388587597512704,...,es,None,NaN,0,False,NaN,web,@espeandgio Miraré de hacer algo con ella. Tam...,False,"{'profile_sidebar_border_color': '5ED4DC', 'de..."


### Splitting the DataFrame to get required rows and creating another DataFrame joining Time Zone DataFrame

In [15]:
#data.iloc[0] # gets the first row of DF
#cnt = rawTweets.id.count() #gets DF count
idSeries = pd.Series(rawTweets['id'])
textSeries = pd.Series(rawTweets['text'])
x = pd.DataFrame(list(rawTweets['user']))
tzSeries = x['time_zone']
timeStampSeries = pd.Series(rawTweets['created_at'])
timeStampSeries = pd.to_datetime(timeStampSeries)

simpleTweets = pd.concat([idSeries, timeStampSeries, textSeries, tzSeries], axis=1)
cleanTweets = pd.merge(simpleTweets, time_zone, how='left', on='time_zone', copy=True)
cleanTweets.head(10)

,id,created_at,text,time_zone,country
0,435388391811997696,2014-02-17 12:20:56,Sabi nung isang tao dito nung shinutdown si ro...,None,NaN
1,435388432211525632,2014-02-17 12:21:06,ROBOCOP B) (@ XXI Summarecon Mal Bekasi) http:...,Bangkok,THAILAND
2,435388451060322304,2014-02-17 12:21:10,Robocop =D ga jadiwess ..,Jakarta,INDONESIA
3,435388458362630144,2014-02-17 12:21:12,Watching Robocop with my baby bro,Pacific Time (US & Canada),UNITED STATES
4,435388456366116865,2014-02-17 12:21:11,أمس شفت Robocop .. طلع أفضل مما توقعته .. فلم ...,Central Time (US & Canada),UNITED STATES
5,435388514851495936,2014-02-17 12:21:25,istrine Alex Murphy cakep banget #RoboCop,Bangkok,THAILAND
6,435388477463474176,2014-02-17 12:21:16,@erraaisya dok dap op cite robocop,Beijing,CHINA
7,435388533872676864,2014-02-17 12:21:30,"Watching RoboCop (with Adila, Muhammad , and 6...",Bangkok,THAILAND
8,435388552738648064,2014-02-17 12:21:34,Watching RoboCop (with Bayu at Puri XXI) — htt...,Jakarta,INDONESIA
9,435388587597512704,2014-02-17 12:21:43,@espeandgio Miraré de hacer algo con ella. Tam...,Madrid,SPAIN


### Cleaning twitter text to remove special characters. Then, splitting twitter text into series of words. Next step is to load the english dictionary which has polarity associated with every word. 

### Then we run a loop to generate sentiment value for every tweet Series.

In [16]:
cleanTextSeries = textSeries.apply(lambda x: re.sub("[^a-zA-Z' ']","",x))
splitTextSeries = cleanTextSeries.apply(lambda x: x.strip().split(' '))

finalData = pd.concat([idSeries, splitTextSeries], axis = 1)
finalData['sentiment'] = 0
finalData['country'] = cleanTweets.country
dictionary = pd.io.parsers.read_csv("C:\\Users\\Harsh\\Documents\\Twitter\\dictionary.csv")
positiveDictionary = dictionary[(dictionary['polarity'] == 'positive')]
negativeDictionary = dictionary[(dictionary['polarity'] == 'negative')]

def calculateSentiment(s):
    count = 0
    for x in s:
        if str(x) in positiveDictionary.string.values:
            count += 1
        elif str(x) in negativeDictionary.string.values:
            count -= 1
    return count

for index, row in finalData.iterrows():
    x = calculateSentiment(finalData.loc[index, "text"])
    finalData.loc[index, "sentiment"] = x

testData = finalData[(finalData['sentiment'] != 0)]
testData.head(10)

,id,text,sentiment,country
10,435388609903202304,"[Idk, if, robocop, is, good, or, not]",1,NETHERLANDS
16,435388722054299648,"[Wondering, if, the, security, guards, at, Leg...",1,UNITED STATES
19,435388742447022081,"[I, want, to, see, the, new, Robocop, just, to...",1,FINLAND
21,435388549505228800,"[RT, BrooklynPRLA, Robocop, actress, abbiecorn...",1,SPAIN
25,435388776848699393,"[Robocop, Robocop, , Action, Figure, with, Spr...",1,UNITED STATES
26,435388793445961728,"[I, kind, of, want, to, see, RoboCop]",2,UNITED KINGDOM
27,435388789574606848,"[RT, odiomistweets, El, remake, de, Robocop, e...",-1,UNITED STATES
30,435388810852323328,"[I, need, to, see, RoboCop, justsaying]",-1,UNITED KINGDOM
33,435388854653030400,"[I, want, to, go, see, Robocop]",1,UNITED KINGDOM
34,435388838890835968,"[OK, I, feel, I, am, getting, a, little, bit, ...",-1,NaN


### Plotting Pie chart based on generated data

In [17]:
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

frame = finalData
frame = frame.drop('text', 1)

result = [0,0,0]
for index, row in frame.iterrows():
    if frame.loc[index, "sentiment"] > 0:
        result[0] += 1
    elif frame.loc[index, "sentiment"] < 0:
        result[1] += 1
    else :
        result[2] += 1

colors =['g','r','b']
labels=['Positive', 'Negative', 'Neutral']

plt.pie(result, colors=colors, labels=labels,  autopct='%.1f%%',counterclock=False, shadow=True)
plt.title('Sentiment analysis')
plt.show()

### Plotting bar graph

In [18]:
newVar = frame.groupby('country').sum().reset_index()
countCountry = newVar.count()

newVar['positive'] = newVar['sentiment'] > 0

ax = newVar['sentiment'].plot(kind='bar', width=.3, align='center', fontsize=12, legend=False,
                                         title ="Sentiment by countries", color=newVar.positive.map({True: 'g',False: 'r'}))

plt.xticks(range(countCountry[0]), newVar['country'] )
ax.set_ylabel('Level of sentiment')
ax.set_xlabel('Countries')
    
plt.show()

In [23]:
from IPython.display import Image
Image(filename='C:\\Users\\Harsh\\Anaconda3\\Notebooks\\CIS_8005_Project\\questions.jpeg')